In [1]:
!pip install pyngrok
!pip install firebase_admin
!pip install diffusers accelerate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install -r requirments.txt

In [17]:
from SegBody import segment_body
from flask import Flask, request, jsonify
from pyngrok import ngrok
from diffusers import AutoPipelineForInpainting, AutoencoderKL
from diffusers.utils import load_image
import torch
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from google.cloud.firestore_v1.base_query import FieldFilter
from PIL import Image
import base64
import io
from classification import closets_color, remove_bg, predict_features
from outfit_recommendation import get_clothes, filter_by_wear_type, get_recommendation

#from flask_ngrok import run_with_ngrok

In [5]:
cred = credentials.Certificate("") # Add our Firebase Credential Certificate .json
firebase_admin.initialize_app(cred, {
    'storageBucket': '' # Bukcet Reference
})
db = firestore.client()

In [6]:
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipeline = AutoPipelineForInpainting.from_pretrained("diffusers/stable-diffusion-xl-1.0-inpainting-0.1", vae=vae, torch_dtype=torch.float16, variant="fp16", use_safetensors=True).to("cuda")
pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin", low_cpu_mem_usage=True)

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

...

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

The config attributes {'decay': 0.9999, 'inv_gamma': 1.0, 'min_decay': 0.0, 'optimization_step': 37000, 'power': 0.6666666666666666, 'update_after_step': 0, 'use_ema_warmup': False} were passed to UNet2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


ip-adapter_sdxl.bin:   0%|          | 0.00/703M [00:00<?, ?B/s]

sdxl_models/image_encoder/config.json:   0%|          | 0.00/2.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.69G [00:00<?, ?B/s]

In [25]:
def get_prompt(clothing_item, color):
  prompt="photorealistic, perfect body, beautiful skin, realistic skin, natural skin"
  negative_prompt="ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin"
  full_top_wears = ['shirt', 'jacket', 'sweater', 'kameez', 'sweat shirt', ]
  full_bottom_wears = ['pant', 'leggings', 'shalwar', 'jeans', 'trousers', 'skirt'] # if kameez then prompt should photorealistic, perfect body, beautiful skin, realistic skin, natural skin, kameez, eastern dress with long sleeves
  half_top_wears = ['t-shirt', 'polo Shirt']
  half_bottom_wear = ['shorts']

  if clothing_item in full_top_wears:
        prompt += f", {color} color {clothing_item}"
        negative_prompt += ", " + ", ".join(full_bottom_wears + half_bottom_wear + half_top_wears)

  elif clothing_item in full_bottom_wears:
        prompt += f", {color} color {clothing_item}"
        negative_prompt += ", " + ", ".join(full_top_wears + half_bottom_wear + half_top_wears)

  elif clothing_item in half_top_wears:
        prompt += f", {color} color {clothing_item}"
        negative_prompt += ", " + ", ".join(half_bottom_wear + full_bottom_wears + full_top_wears)

  elif clothing_item in half_bottom_wear:
        prompt += f", {color} color {clothing_item}"
        negative_prompt += ", " + ", ".join(half_top_wears + full_bottom_wears + full_top_wears)



  return prompt, negative_prompt


In [8]:
def virtual_try_on(img, clothing, prompt, negative_prompt, ip_scale=1.0, strength=0.99, guidance_scale=7.5, steps=100):
    _, mask_img = segment_body(img, face=False)
    pipeline.set_ip_adapter_scale(ip_scale)
    images = pipeline(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=img,
        mask_image=mask_img,
        ip_adapter_image=clothing,
        strength=strength,
        guidance_scale=guidance_scale,
        num_inference_steps=steps,
    ).images
    return images[0]

In [9]:
def get_clothing_item(item_id):

    clothe_ref = db.collection('clothes')
    clothe = clothe_ref.where(filter=FieldFilter('clotheId', '==', item_id)).stream()
    for doc in clothe:
        return doc.to_dict()
    return None

In [26]:
app = Flask(__name__)
token = "" # Add your ngrok Authentication Token
ngrok.set_auth_token(token)
public_url = ngrok.connect(5000).public_url



In [27]:
@app.route('/virtual_try_on', methods=['GET'])
def try_on():
    bottom_wear_id = request.args.get('bottom_wear')
    top_wear_id = request.args.get('top_wear')
    user_image = request.files['image']
    file_name = 'user_image'
    user_image = Image.open(user_image)
    user_image.save(file_name + '.jpg')
    print('User Image saved')
    items = [bottom_wear_id, top_wear_id]

    for item in items:
        clothe = get_clothing_item(item)
        #clothe_dict = clothe.to_dict()
        pos_prompt, neg_prompt = get_prompt(clothe['Article'].lower(), clothe['color'])
        clothe_image = load_image(clothe['image']).convert("RGB")
        user_image = virtual_try_on(img=user_image, clothing=clothe_image, prompt=pos_prompt, negative_prompt=neg_prompt)
        print("Item used")


    print('virtual try on completed sending image')
    buffer = io.BytesIO()
    user_image.save(buffer, format="JPEG")
    buffer.seek(0)
    try_on_image = base64.b64encode(buffer.getvalue()).decode('utf-8')
    print('ouput saved')

    response = {
        'try_on_image': try_on_image
    }
    print('Response Sended')
    return jsonify(response)

In [ ]:
@app.route('/classify', methods=['GET'])
def classify():
    image_file = request.files['image']
    img_bg = remove_bg(image_file)
    color = closets_color("bgremoved.png")
    features = predict_features(image_file)
    img = base64.b64encode(img_bg).decode('utf-8')

    response = {
        'background_removed_image': img,
        'color': color,
        'features': features
    }
    print('Sending Response')
    return jsonify(response)

In [ ]:
@app.route('/recommendation', methods=['GET'])
def recommend_outfit():
    user_id = request.args.get('user_id')
    venue = request.args.get('venue')
    temperature = int(request.args.get('temperature'))
    event = request.args.get('event')
    filtered_clothes = get_clothes(user_id, temperature, event, venue)
    top_wear, bottom_wear, foot_wear = filter_by_wear_type(filtered_clothes)
    recommended_outfits = get_recommendation(top_wear, bottom_wear, foot_wear)
    return jsonify(recommended_outfits)

In [28]:
print(f'To access this url: {public_url}')
print('for virtual try on add to the end of url:  /virtual_try_on')
print('for virtual try on add to the end of url:  /classify')
print('for virtual try on add to the end of url:  /recommendation')
app.run()

https://5268-34-125-45-47.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Image saved


/usr/local/lib/python3.10/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


  0%|          | 0/99 [00:00<?, ?it/s]

...

  0%|          | 0/99 [00:00<?, ?it/s]

Item used


  0%|          | 0/99 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [09/Aug/2024 14:49:28] "GET /virtual_try_on?bottom_wear=TgEJRUVKAsYM5LFw9pJj&top_wear=UDW9PRujGn14bsPkOU3u HTTP/1.1" 200 -


Item used
virtual try on completed sending image
ouput saved
Response Sended


In [ ]:
bottomwear = "5JdgxLTo5ajPFVOwYfHW"
topwear = "LaAFHZrOD7EExTKQDSQq"